In [ ]:
!pip install turicreate

     |████████████████████████████████| 92.0 MB 6.3 kB/s 
     |████████████████████████████████| 3.5 MB 36.3 MB/s 
     |████████████████████████████████| 86.4 MB 26 kB/s 
     |████████████████████████████████| 322 kB 64.7 MB/s 
     |████████████████████████████████| 3.6 MB 33.4 MB/s 
     |████████████████████████████████| 18.3 MB 83 kB/s 
     |████████████████████████████████| 50 kB 5.4 MB/s 
     |████████████████████████████████| 20.1 MB 1.4 MB/s 
     |████████████████████████████████| 449 kB 50.3 MB/s 
     |████████████████████████████████| 2.9 MB 18.9 MB/s 
     |████████████████████████████████| 3.8 MB 36.0 MB/s 
  Created wheel for prettytable: filename=prettytable-0.7.2-py3-none-any.whl size=13714 sha256=5eb49d34e2b43cb7689f669cb2d7342381147d9f5e57a5be3e3e3a8914ef5acb
  Stored in directory: /root/.cache/pip/wheels/b2/7f/f6/f180315b584f00445045ff1699b550fa895d09471337ce21c6
  Created wheel for resampy: filename=resampy-0.2.1-py3-none-any.whl size=320860 sha256=a235e4a84a7

In [ ]:
# link of turicreate library https://github.com/apple/turicreate
import turicreate as tc # Apple open source library Turi Create that simplifies the development of custom machine learning models.
import pandas as pd

Download our data (OPTIONAL, can skip if running locally)

In [ ]:
import gdown
books_raw_url = 'https://drive.google.com/file/d/1TcFgTOCoBBqfEwrSbhYqunu5qYDnzxSH/view?usp=sharing'
books_url     ='https://drive.google.com/uc?id=' + books_raw_url.split('/')[-2] #Do not change
gdown.download(books_url, 'books.csv',quiet=False)

ratings_raw_url = 'https://drive.google.com/file/d/12noQhcRgAYBUNcbhG2Z3fuNWKX0EebMA/view?usp=sharing'
ratings_url     ='https://drive.google.com/uc?id=' + ratings_raw_url.split('/')[-2] #Do not change
gdown.download(ratings_url, 'ratings.csv',quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1TcFgTOCoBBqfEwrSbhYqunu5qYDnzxSH
To: /content/books.csv
100%|██████████| 3.29M/3.29M [00:00<00:00, 60.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=12noQhcRgAYBUNcbhG2Z3fuNWKX0EebMA
To: /content/ratings.csv
100%|██████████| 72.1M/72.1M [00:01<00:00, 60.0MB/s]


'ratings.csv'

## Data Preprocessing

In [ ]:
df_books = pd.read_csv("books.csv")
print(df_books.shape)
df_books.head()

(10000, 23)


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [ ]:
df_ratings = pd.read_csv("ratings.csv")
print(df_ratings.shape)
df_ratings.head()

(5976479, 3)


,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [ ]:
train_data = tc.SFrame(df_ratings)  # Data needs to be converted to SFrame format

In [ ]:
# Used for quick visualisation and data exploration
train_data.show()

Materializing SFrame

## Train/Test Split

In [ ]:
# 80/20 Train/Test Split
train, test = tc.recommender.util.random_split_by_user(train_data, user_id='user_id', item_id='book_id', item_test_proportion=0.2)

In [ ]:
len(train), len(test)

(5954292, 22187)

# Item-Item Similarity Recommender

In [ ]:
# training the model
item_similarity_model = tc.item_similarity_recommender.create(train, user_id='user_id', item_id='book_id', target='rating', similarity_type='cosine')

Preparing data set.

Data has 5954292 observations with 53424 users and 10000 items.

Data prepared in: 6.81771s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 21.91s                              | 100              | 10000           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 23.2954s

In [ ]:
# making top 10 recommendations for the first three users
item_similarity_recomm = item_similarity_model.recommend(users=[1,2,3], k=10)
item_similarity_recomm.print_rows(num_rows=30)

+---------+---------+---------------------+------+
| user_id | book_id |        score        | rank |
+---------+---------+---------------------+------+
|    1    |    8    |  0.572478973457956  |  1   |
|    1    |    5    |  0.5503190603011694 |  2   |
|    1    |    1    | 0.49530970540821045 |  3   |
|    1    |    15   |  0.4640308674584087 |  4   |
|    1    |    14   | 0.45124688056799084 |  5   |
|    1    |    2    |  0.4442195683463007 |  6   |
|    1    |   118   |  0.4399895178966033 |  7   |
|    1    |    30   |  0.3876202590445168 |  8   |
|    1    |   195   |  0.3818955732207013 |  9   |
|    1    |    26   |  0.3566504416302738 |  10  |
|    2    |    4    |  0.7213789022885836 |  1   |
|    2    |    14   |  0.6848287224769593 |  2   |
|    2    |    7    |  0.6129147456242487 |  3   |
|    2    |    1    |  0.6005118342546316 |  4   |
|    2    |    58   |  0.5398400774368873 |  5   |
|    2    |    19   |  0.5013636304781988 |  6   |
|    2    |    48   |  0.498170

In [ ]:
# Item-Item Similarity Recommender Performance
item_similarity_model_performance = item_similarity_model.evaluate(test)

recommendations finished on 1000/1000 queries. users per second: 2067.12


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    |        0.452        | 0.021721416015350527 |
|   2    |        0.405        | 0.03885322610527315  |
|   3    | 0.37633333333333363 |  0.0543022547133502  |
|   4    |  0.3490000000000001 | 0.06745078388090187  |
|   5    |        0.3296       | 0.07944853555904154  |
|   6    | 0.30966666666666703 |  0.0893911585844669  |
|   7    | 0.29785714285714276 |  0.1003307812284129  |
|   8    |  0.2852500000000001 |  0.1093036621491776  |
|   9    |  0.2749999999999998 | 0.11846972487051183  |
|   10   | 0.26689999999999997 | 0.12757121557466436  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.9027380105470764

Per User RMSE (best)
+---------+-------------------+-------+
| user_id |        rmse       | count |
+-----

In [ ]:
# The RMSE value around 3.90 for item-item similarity recommender
item_similarity_model_performance['rmse_overall']

3.9027380105470764

In [ ]:
# Making top 10 recommendation for user 1 and store in dataframe 
user1_recomm = item_similarity_model.recommend(users=[1], k=10)
df_user1_recomm = user1_recomm.to_dataframe()

In [ ]:
df_user1_recomm = df_user1_recomm.merge(df_books[['book_id', 'title', 'isbn', 'authors']])

In [ ]:
df_user1_recomm

,user_id,book_id,score,rank,title,isbn,authors
0,1,8,0.572479,1,The Catcher in the Rye,316769177,J.D. Salinger
1,1,5,0.550319,2,The Great Gatsby,743273567,F. Scott Fitzgerald
2,1,1,0.495310,3,"The Hunger Games (The Hunger Games, #1)",439023483,Suzanne Collins
3,1,15,0.464031,4,The Diary of a Young Girl,553296981,"Anne Frank, Eleanor Roosevelt, B.M. Mooyaart-D..."
4,1,14,0.451247,5,Animal Farm,452284244,George Orwell
5,1,2,0.444220,6,Harry Potter and the Sorcerer's Stone (Harry P...,439554934,"J.K. Rowling, Mary GrandPré"
6,1,118,0.439990,7,The Joy Luck Club,143038095,Amy Tan
7,1,30,0.387620,8,Gone Girl,297859382,Gillian Flynn
8,1,195,0.381896,9,The Guernsey Literary and Potato Peel Pie Society,385340990,"Mary Ann Shaffer, Annie Barrows"
9,1,26,0.356650,10,"The Da Vinci Code (Robert Langdon, #2)",307277674,Dan Brown


# Matrix Factorization Recommender Collaborative Filtering

### Build a Model Based or Matrix Based Factorization Recommender 

In [ ]:
# training the model
factorization_model = tc.factorization_recommender.create(train, user_id='user_id', item_id='book_id', target='rating')

Preparing data set.

Data has 5954292 observations with 53424 users and 10000 items.

Data prepared in: 6.87251s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.529269

Final training RMSE: 0.726886

In [ ]:
# making top 10 recommendations for the first three users
factorization_recomm = factorization_model.recommend(users=[1,2,3],k=10)
factorization_recomm.print_rows(num_rows=25)

+---------+---------+--------------------+------+
| user_id | book_id |       score        | rank |
+---------+---------+--------------------+------+
|    1    |   5584  | 5.429468139145894  |  1   |
|    1    |   9732  | 5.3844256243448685 |  2   |
|    1    |   8641  | 5.296988590691609  |  3   |
|    1    |   9553  | 5.2960806927423905 |  4   |
|    1    |   9220  |  5.28712986323647  |  5   |
|    1    |   1233  | 5.169709309075398  |  6   |
|    1    |   1476  | 5.160301669571919  |  7   |
|    1    |   9625  | 5.157415493462605  |  8   |
|    1    |   1289  | 5.153903826211018  |  9   |
|    1    |   1033  | 5.142230614159627  |  10  |
|    2    |   8326  | 5.5747612259130905 |  1   |
|    2    |   9537  |  5.56357081148438  |  2   |
|    2    |   3628  | 5.551012947295232  |  3   |
|    2    |   6527  | 5.491216494772954  |  4   |
|    2    |   2767  | 5.474042370055241  |  5   |
|    2    |   8666  | 5.471668793890996  |  6   |
|    2    |   2244  | 5.455030991766972  |  7   |


In [ ]:
# Matrix Factorization Recommender Performance
factorization_model_performance = factorization_model.evaluate(test)

recommendations finished on 1000/1000 queries. users per second: 2196.06


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |          0.0          |          0.0           |
|   2    | 0.0010000000000000002 | 7.703703703703701e-05  |
|   3    | 0.0009999999999999994 | 0.0001064488017429194  |
|   4    | 0.0010000000000000002 | 0.00014093156036360904 |
|   5    | 0.0007999999999999999 | 0.00014093156036360904 |
|   6    | 0.0011666666666666663 | 0.00023914584607789474 |
|   7    | 0.0011428571428571427 | 0.0002808125127445615  |
|   8    | 0.0009999999999999994 | 0.0002808125127445615  |
|   9    | 0.0013333333333333333 | 0.0004389368482602012  |
|   10   |         0.0013        | 0.00047227018159353443 |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.843823101713811

Per User RMSE (best)
+---------+--------------------

In [ ]:
# The RMSE value around 0.84 for matrix factorization recommender
factorization_model_performance['rmse_overall']

0.843823101713811

In [ ]:
# Making top 10 recommendation for user 1 and store in dataframe 
user1_recomm_fact_model = factorization_model.recommend(users=[1], k=10)
df_user1_recomm_fact_model = user1_recomm_fact_model.to_dataframe()

In [ ]:
df_user1_recomm_fact_model = df_user1_recomm_fact_model.merge(df_books[['book_id', 'title', 'isbn', 'authors']])

In [ ]:
df_user1_recomm_fact_model

,user_id,book_id,score,rank,title,isbn,authors
0,1,5584,5.429468,1,"Avoiding Commitment (Avoiding, #1)",NaN,K.A. Linde
1,1,9732,5.384426,2,Economics in One Lesson: The Shortest & Surest...,517548233,Henry Hazlitt
2,1,8641,5.296989,3,"Kings Rising (Captive Prince, #3)",174348495X,C.S. Pacat
3,1,9553,5.296081,4,"Attack on Titan: No Regrets, Volume 01",1612629415,"Hajime Isayama, Hikaru Suruga, Gan Sunaaku"
4,1,9220,5.287130,5,On Becoming Baby Wise: Giving Your Infant the ...,1932740082,Gary Ezzo
5,1,1233,5.169709,6,The Vile Village (A Series of Unfortunate Even...,60566221,"Lemony Snicket, Brett Helquist"
6,1,1476,5.160302,7,The Slippery Slope (A Series of Unfortunate Ev...,64410137,"Lemony Snicket, Brett Helquist"
7,1,9625,5.157415,8,"Healthy Sleep Habits, Happy Child",449004023,Marc Weissbluth
8,1,1289,5.153904,9,The Ersatz Elevator (A Series of Unfortunate E...,60566213,"Lemony Snicket, Brett Helquist"
9,1,1033,5.142231,10,The Wide Window (A Series of Unfortunate Event...,64407683,"Lemony Snicket, Brett Helquist"


# Compare both Models

In [ ]:
model_performance = tc.recommender.util.compare_models(test, [item_similarity_model, factorization_model])

PROGRESS: Evaluate model M0


recommendations finished on 1000/1000 queries. users per second: 2018.31


Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |        0.452        | 0.02172141601535053 |
|   2    |        0.405        | 0.03885322610527316 |
|   3    | 0.37633333333333346 | 0.05430225471335021 |
|   4    | 0.34899999999999987 | 0.06745078388090181 |
|   5    |  0.3295999999999999 | 0.07944853555904155 |
|   6    | 0.30966666666666653 | 0.08939115858446689 |
|   7    | 0.29785714285714293 | 0.10033078122841288 |
|   8    |  0.2852499999999999 | 0.10930366214917761 |
|   9    |  0.2750000000000002 |  0.1184697248705119 |
|   10   | 0.26689999999999986 | 0.12757121557466433 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 3.9027380105470764

Per User RMSE (best)
+---------+-------------------+-------+
| user_id |        rmse       | count |
+---------+---------

recommendations finished on 1000/1000 queries. users per second: 2047.73


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |          0.0          |          0.0           |
|   2    | 0.0010000000000000002 | 7.703703703703704e-05  |
|   3    | 0.0009999999999999994 | 0.0001064488017429194  |
|   4    | 0.0010000000000000002 |  0.000140931560363609  |
|   5    | 0.0007999999999999999 |  0.000140931560363609  |
|   6    | 0.0011666666666666665 | 0.0002391458460778947  |
|   7    | 0.0011428571428571427 | 0.0002808125127445615  |
|   8    | 0.0009999999999999994 | 0.0002808125127445615  |
|   9    | 0.0013333333333333335 | 0.00043893684826020135 |
|   10   | 0.0013000000000000004 | 0.0004722701815935344  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.843823101713811

Per User RMSE (best)
+---------+--------------------

**Matrix Factorization recommender gave a better RMSE of 0.84 than item-item similarity recommender gave RMSE of 3.90**